In [4]:
from openai import OpenAI
import pandas as pd
import json
import ast
from dotenv import load_dotenv
from tqdm import tqdm
import os

load_dotenv()

my_key = os.getenv("MY_API_KEY")

In [5]:
client = OpenAI(api_key=my_key)

In [6]:
batch_size = 50   # how many rows per API call
labels = []

In [ ]:
MODEL = "gpt-4o-mini"
df = pd.read_csv("../Data/ishowspeed_chat_messages.csv")

for i in tqdm(range(0, len(df), batch_size), desc="Batch labeling"):
    batch_texts = df["message"].iloc[i:i+batch_size].tolist()

    # Construct the batch prompt
    prompt = "Classify each of the following chat messages as Offensive (1) or Not Offensive (0).\n"
    prompt += "Return only a Python-style list of 0s and 1s, in the same order.\n\n"

    for j, text in enumerate(batch_texts, start=1):
        prompt += f"{j}. {text}\n"

    # Call OpenAI
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a moderation classifier."},
            {"role": "user", "content": prompt}
        ],
        temperature=0,
    )

    # Parse the output
    output = response.choices[0].message.content.strip()
    try:
        # batch_labels = eval(output)  # risky but works if model outputs [0,1,0,...]
        batch_labels = ast.literal_eval(output)  # safer
    except:
        # fallback if parsing fails
        batch_labels = [int(x) for x in output.replace("[","").replace("]","").split(",")]

    labels.extend(batch_labels)

Batch labeling: 100%|██████████| 2/2 [00:09<00:00,  4.91s/it]


In [11]:
df["label"] = labels
output_name = "../Data/twitch_chat_labeled_by_GPT.csv"
df.to_csv(output_name, index=False)

In [12]:
df[df['label'] == 1]

,message,label
60,nigga you need some pussy,1
